In [1]:
%load_ext autoreload

In [2]:
from config import Config
from pprint import pprint, pformat
from logger import model_logger
log = model_logger.getLogger('main')
log.setLevel(Config.Log.MODEL.level)

2018-01-29 11:27:47,145:root:INFO   :           getLogger:: creating logger for main under MODEL


In [3]:
%autoreload 2
from trainer import Trainer, Feeder, Predictor
from datafeed import DataFeed
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
import torch

2018-01-29 11:27:47,211:root:INFO   :           getLogger:: creating logger for main under TRAINER
2018-01-29 11:27:47,428:root:INFO   :           getLogger:: creating logger for main under DATAFEED


In [4]:
import csv
train_dataset = csv.reader(open('dataset/train.csv'))
test_dataset = csv.reader(open('dataset/test.csv'))

In [5]:
from collections import namedtuple
Sample = namedtuple('Sample', ['id','comment_text',
                               'toxic','severe_toxic','obscene',
                               'threat','insult','identity_hate'])

### Unicode to ascii text

In [6]:
import unicodedata
train_datapoints = []
for i in list(train_dataset)[1:]:
    _id, c, t, st, o, t, ins, ih = i
    t, st, o, t, ins, ih = (int(_) for _ in [t, st, o, t, ins, ih])
    c = unicodedata.normalize('NFD', c).encode('ascii','ignore').decode()
    train_datapoints.append(Sample(_id, c, t, st, o, t, ins, ih))

test_datapoints = []
for i in list(test_dataset)[1:]:
    _id, c = i
    c = unicodedata.normalize('NFD', c).encode('ascii','ignore').decode()
    test_datapoints.append(Sample(_id, c, 0, 0, 0, 0, 0, 0))

len(train_datapoints), len(test_datapoints)

(159571, 153164)

In [7]:
#train_datapoints = train_datapoints[:1000]

In [8]:
test_datapoints[1000:1010]

[Sample(id='01ac9982edae9977', comment_text='" \n\n Dear ,Welcome to Wikipedia!Unfortunately, using your e-mail address as your username is not a good idea. Wikipedia content is extensively copied and the site itself is one of the most visited sites in the world. Any edit you make on Wikipedia will have your username attached to it, and using your email address will make you a tempting target for spammers. We recommend that you change your username at Wikipedia:Changing username in order to prevent abuse.If you need any help, simply contact me on my talk page, or go to Wikipedia:Help desk. Another option is to place  on your own talk page, and someone will come shortly to help. Remember to sign your posts on talk pages with four tildes (~~~~). Again, welcome! -  at "', toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0),
 Sample(id='01ac9ea1271d7409', comment_text='Asshole, your dirty MF people tries to steal our glory and heavy persian (tajik) history by claiming 

## Build vocabulary

#### buils INPUT_VOCAB

In [9]:
%timeit
from nltk import word_tokenize
from tqdm import tqdm
datapoints = train_datapoints
INPUT_VOCAB = [word for dp in tqdm(datapoints) for word in word_tokenize(dp.comment_text)]
INPUT_VOCAB = list(set(INPUT_VOCAB))
len(INPUT_VOCAB)

100%|██████████| 159571/159571 [01:28<00:00, 1800.43it/s]


299974

In [10]:
INPUT_VOCAB[:10]

['insane',
 'ass/face',
 '*t',
 'FLUID',
 'members/archive1',
 'cut-out.',
 "'TS",
 'PED-OH-PHILE',
 'Plainlist',
 'AMAZING.']

In [11]:
OUTPUT_VOCAB = ['toxic','severe_toxic','obscene', 'threat','insult','identity_hate']
INPUT_VOCAB = ['<<PAD>>', '<<UNK>>'] + list(set(INPUT_VOCAB + OUTPUT_VOCAB))

#### builds WORD_INDEX

In [12]:
WORD_INDEX = {w: i for i, w in enumerate(INPUT_VOCAB)}
OUTPUT_IDS = [WORD_INDEX[i] for i in OUTPUT_VOCAB]

In [13]:
sorted(list(WORD_INDEX.items()), key=lambda x: x[1])[:10], WORD_INDEX['<<PAD>>'], INPUT_VOCAB[0]

([('<<PAD>>', 0),
  ('<<UNK>>', 1),
  ('insane', 2),
  ('FLUID', 3),
  ('ass/face', 4),
  ('*t', 5),
  ('members/archive1', 6),
  ('cut-out.', 7),
  ("'TS", 8),
  ('PED-OH-PHILE', 9)],
 0,
 '<<PAD>>')

## tests INPUTVOCAB and WORD_INDEX mapping

In [14]:
import random
_i = train_datapoints[random.choice(range(len(train_datapoints)))]
print(_i.comment_text)
print("""


""")
print(
      ' '.join( [INPUT_VOCAB[i] for i in 
                 [WORD_INDEX[j] for j in word_tokenize(_i.comment_text)]]
              )
     )

Note to Eclecticology: Hum, you just brought to my attention a naming conflict for two cities named Paris over at List of places and things named Paris. Are their other such conflicts within Canada? And if so are most of these between Ontario and the rest of Canada? The city naming convention was debated and worked out on the wikipedia mailing list and in several other places  it would have been nice to know of any possible systemic internal naming conflicts within Canada. Maybe this is one of the few exceptions that we can disambiguate on a case-by-case basis What is the population of each of these Canadian Paris? Is it at all likely that either of these will become encyclopedia articles? maveric149




Note to Eclecticology : Hum , you just brought to my attention a naming conflict for two cities named Paris over at List of places and things named Paris . Are their other such conflicts within Canada ? And if so are most of these between Ontario and the rest of Canada ? The city namin

## Baseline model

In [15]:
class Model(nn.Module):
    def __init__(self, Config, input_vocab_size, output_vocab_size):
        super(Model, self).__init__()
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.hidden_dim = Config.hidden_dim

        self.embed = nn.Embedding(self.input_vocab_size, self.hidden_dim)
        self.encode = nn.GRUCell(self.hidden_dim, self.hidden_dim)

        self.classify = [nn.Linear(self.hidden_dim, 2)
                         for i in range (self.output_vocab_size)]

        self.log = model_logger.getLogger('model')
        self.log.setLevel(logging.INFO)
        if Config.cuda:
            self.cuda()
            [i.cuda() for i in self.classify]
        
    def init_hidden(self, batch_size):
        ret = torch.zeros(batch_size, self.hidden_dim)
        if Config().cuda: ret = ret.cuda()
        return Variable(ret)
    
    def forward(self, seq):
        seq = Variable(torch.LongTensor(seq))
        if Config().cuda: seq = seq.cuda()
        batch_size = seq.size()[0]
        self.log.debug('{} seq size: {}'.format(type(seq.data), seq.size()))
        seq_emb = self.embed(seq).transpose(1,0)
        output = self.init_hidden(batch_size)
        for token_emb in seq_emb:
            self.log.debug('token_emb := {}'.format(token_emb))
            self.log.debug('output := {}'.format(output))
            output = self.encode(token_emb, output)
                    
        self.log.debug('output := {}'.format(output))
    
        ret = torch.stack([F.softmax(classify(output), dim=-1) 
                           for classify in self.classify])
        self.log.debug('ret := {}'.format(ret))

        self.log.debug('ret size: {}'.format(ret.size()))

        return ret

### Batching utils

In [16]:
import numpy as np
def seq_maxlen(seqs):
    return max([len(seq) for seq in seqs])

PAD = WORD_INDEX[INPUT_VOCAB[0]]
print(PAD)
def pad_seq(seqs, maxlen=0, PAD=PAD):
    if type(seqs[0]) == type([]):
        maxlen = maxlen if maxlen else seq_maxlen(seqs)
        def pad_seq_(seq):
            return seq + [PAD]*(maxlen-len(seq))
        seqs = [ pad_seq_(seq) for seq in seqs ]
    return seqs

def batchop(datapoints, *args, **kwargs):
    indices = [d.id for d in datapoints]
    seq   = pad_seq([ [WORD_INDEX[w] for w in word_tokenize(d.comment_text)]
                     for d in datapoints])
    target = [(d.toxic, d.severe_toxic, d.obscene, d.threat, d.insult, d.identity_hate)
              for d in datapoints]
    seq, target = np.array(seq), np.array(target)
    return indices, (seq, ), (target,)

0


## Attention based model

In [17]:
class AttModel(nn.Module):
    def __init__(self, Config, input_vocab_size, output_vocab_size):
        super(AttModel, self).__init__()
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.hidden_dim = Config.hidden_dim

        self.embed = nn.Embedding(self.input_vocab_size, self.hidden_dim)
        self.encode = nn.GRUCell(self.hidden_dim, self.hidden_dim)
        self.attend = nn.Parameter(torch.FloatTensor(self.hidden_dim, self.hidden_dim))

        self.classify = nn.Linear(self.hidden_dim, 2)
        self.log = model_logger.getLogger('model')
        self.size_log = self.log.getLogger('size')
        self.log.setLevel(logging.INFO)
        self.size_log.setLevel(logging.DEBUG)
        if Config.cuda:
            self.cuda()
            
    def logsize(self, tensor, name=''):
        self.size_log.debug('{} <- {}'.format(tensor.size(), name))
        
    def init_hidden(self, batch_size):
        ret = torch.zeros(batch_size, self.hidden_dim)
        if Config().cuda: ret = ret.cuda()
        return Variable(ret)
    
    def forward(self, seq, classes=OUTPUT_IDS):
        seq = Variable(torch.LongTensor(seq))
        classes = Variable(torch.LongTensor(classes))
        if Config().cuda: 
            seq = seq.cuda()
            classes = classes.cuda()
            
        batch_size = seq.size()[0]
        self.log.debug('{} seq size: {}'.format(type(seq.data), seq.size()))
        seq_emb = self.embed(seq).transpose(1,0)                  ;self.logsize(seq_emb, 'seq_emb')
        seq_repr = []
        output = self.init_hidden(batch_size)                     ;self.logsize(output, 'output')
        for token_emb in seq_emb:
            self.log.debug('token_emb := {}'.format(token_emb))
            self.log.debug('output := {}'.format(output))
            output = self.encode(token_emb, output)               ;self.logsize(output, 'output')
            seq_repr.append(output)

        seq_repr = torch.stack(seq_repr).transpose(1,0)           ;self.logsize(seq_repr, 'seq_repr')
        outputs = []
        attend = self.attend
        self.logsize(attend, 'attend')
        for class_ in classes:
            class_emb = self.embed(class_)                  ;self.logsize(class_emb, 'class_emb')
            attn = torch.mm(class_emb, attend).unsqueeze(0) ;self.logsize(attn, 'attn')
            attn = attn.expand_as(seq_repr) ;self.logsize(attn, 'attn')
            attended_outputs = torch.bmm(attn, seq_repr)                
        
            self.logsize(attended_outputs, 'attended_outputs')
            output = attended_output.expand_as(seq_repr) * seq_repr
            self.logsize(output, 'output')

            output = output.sum(0).squeeze()                ;self.logsize(output, 'output')
            output = self.classify(output)                  ;self.logsize(output, 'output')
            output = F.softmax(output)
            outputs.append(output)
            
        ret = torch.stack(outputs)
        return ret

## Loss and accuracy function

In [18]:
def loss(output, target, loss_function=nn.NLLLoss(), *args, **kwargs):
    loss = 0
    target = Variable(torch.LongTensor(target[0]))
    if Config().cuda: target = target.cuda()
    output = output.transpose(1,0)
    batch_size = output.size()[0]
    for i, t in zip(output, target):
        log.debug('i, o sizes: {} {}'.format(i, t))
        loss += loss_function(i, t.squeeze()).mean()
        log.debug('loss size: {}'.format(loss))

    return loss/batch_size

def accuracy(output, target, *args, **kwargs):
    accuracy = 0
    target = Variable(torch.LongTensor(target[0]))
    if Config().cuda: target = target.cuda()
    output = output.transpose(1,0)
    batch_size = output.size()[0]
    class_size = output.size()[1]    
    
    for i, t in zip(output, target):
        correct = (i.max(dim=1)[1] == t).sum()
        accuracy += correct/class_size
        
    return (accuracy/batch_size).data[0]
    

### repr_function to build human readable output from model

In [19]:
from IPython.display import HTML
from IPython.display import display
def repr_function(output, feed, batch_index):
    results = []
    output = output.transpose(1,0)
    indices, (seq,), (classes,) = feed.nth_batch(batch_index)
    print(output.size(), len(indices), len(seq), len(classes))
    for i, o, s, c in zip(indices, output, seq, classes):
        orig_s = feed.data_dict[i].comment_text
        s = [INPUT_VOCAB[i] for i in s]
        s = ' '.join(s)
        results.append([orig_s, s] + list(c))
        o = o.max(dim=1)[1]
        results.append([' ', '  '] + o.data.cpu().numpy().tolist())
    return results

In [20]:
_train_datapoints = train_datapoints

In [ ]:
import logging
log.setLevel(logging.INFO)

## Experiment on model 1

In [ ]:
import random
def  experiment(epochs=10, checkpoint=1):
    model =  Model(Config(), len(INPUT_VOCAB), len(OUTPUT_VOCAB))
    if Config().cuda:  model = model.cuda()
        
    split_index = int( len(train_datapoints) * 0.85 )
    train_feed = DataFeed(train_datapoints[:split_index], batchop=batchop, batch_size=128)
    test_feed = DataFeed(train_datapoints[split_index:], batchop=batchop, batch_size=120)

    trainer = Trainer(model=model, loss_function=loss, accuracy_function=accuracy, 
                    checkpoint=checkpoint, epochs=epochs,
                    feeder = Feeder(train_feed, test_feed))

    predictor = Predictor(model=model, repr_function=repr_function, feed=test_feed)
    output, results = predictor.predict(random.choice(range(test_feed.num_batch)))

    for e in range(10000):
        output, results = predictor.predict(random.choice(range(test_feed.num_batch)))
        display(HTML(results._repr_html_()))
        trainer.train()
        
experiment()

2018-01-29 11:29:26,765:root:INFO   :           getLogger:: creating logger for model under MODEL


torch.Size([120, 6, 2]) 120 120 120
torch.Size([120, 6, 2]) 120 120 120


signed hello world oh and i would like u to apoligize for your rude and unwarrneted behavour end...,signed hello world oh and i would like u to apoligize for your rude and unwarrneted behavour end ... <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0,0,0,1,1,0
""" I have removed the helpme tag for now, don't hesitate to put it back with your question! talk ""","`` I have removed the helpme tag for now , do n't hesitate to put it back with your question ! talk `` <> <> <> <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0,0,0,1,1,0
YOU ARE REALLY ANNOYING!!!! gO SCREW YOUR LESBIAN FRIENDS!!! nEED SOME ICE?? YOU JUST GOT BURNED!!!,YOU ARE REALLY ANNOYING ! ! ! ! gO SCREW YOUR LESBIAN FRIENDS ! ! ! nEED SOME ICE ? ? YOU JUST GOT BURNED ! ! ! <> <> <> <> <> <> <> <> <>,0,0,1,0,1,1
,,0,0,0,1,1,0
""" Well, the page where you'd do this is WP:RFA. I suggest you get more experience first. Charge! ""","`` Well , the page where you 'd do this is WP : RFA . I suggest you get more experience first . Charge ! `` <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0,0,0,1,1,0
"The film looks awesome, but my Dad said the original was much different. Im seeing it this weekend.","The film looks awesome , but my Dad said the original was much different . Im seeing it this weekend . <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0,0,0,1,1,0
0 | id = ar | info = This user does not understand Arabic. }}{{userbox-level | level =,0 | id = ar | info = This user does not understand Arabic . } } { { userbox-level | level = <> <> <> <> <> <> <> <> <> <> <> <> <> <>,0,0,0,0,0,0


2018-01-29 11:29:29,424:TRAINER.main:CRITICAL:               train:: memory consumed : 2462363648
 83%|████████▎ | 882/1059 [04:25<01:43,  1.71it/s]

## Experiment on model using attention

In [ ]:
import random
def  experiment(epochs=10, checkpoint=1):
    model =  Model(Config(), len(INPUT_VOCAB), len(OUTPUT_VOCAB))
    if Config().cuda:  model = model.cuda()
        
    split_index = int( len(train_datapoints) * 0.85 )
    train_feed = DataFeed(train_datapoints[:split_index], batchop=batchop, batch_size=128)
    test_feed = DataFeed(train_datapoints[split_index:], batchop=batchop, batch_size=120)

    trainer = Trainer(model=model, loss_function=loss, accuracy_function=accuracy, 
                    checkpoint=checkpoint, epochs=epochs,
                    feeder = Feeder(train_feed, test_feed))

    predictor = Predictor(model=model, repr_function=repr_function, feed=test_feed)
    output, results = predictor.predict(random.choice(range(test_feed.num_batch)))

    for e in range(10000):
        output, results = predictor.predict(random.choice(range(test_feed.num_batch)))
        display(HTML(results._repr_html_()))
        trainer.train()
        
experiment()

In [ ]:
dummy_feed = DataFeed(train_datapoints[:100], batchop=batchop, batch_size=1)
indices, (seq,), (target,) = dummy_feed.nth_batch(random.choice(range(dummy_feed.num_batch)))
print(dummy_feed.data_dict[indices[0]])
print([INPUT_VOCAB[i] for i in seq[0]])